# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_key import g_key

### Store Part I results into DataFrame

In [14]:
#import csv and print dataframe
weather_data = pd.read_csv("..\Output\weather_data.csv")
weather_data

,City,Lat,Lon,Maximum Temp,Humidity,Cloudiness,Wind,Country,Date
0,ushuaia,-54.8000,-68.3000,51.80,76,40,18.41,AR,1612657769
1,husavik,66.0449,-17.3389,32.00,74,75,3.44,IS,1612657742
2,saint-philippe,-21.3585,55.7679,78.80,73,75,6.91,RE,1612657769
3,rikitea,-23.1203,-134.9692,78.67,69,100,9.84,PF,1612657769
4,pitimbu,-7.4706,-34.8086,78.80,83,20,4.61,BR,1612657770
...,...,...,...,...,...,...,...,...,...
558,taree,-31.9000,152.4667,80.60,65,40,10.36,AU,1612657862
559,ipoti,7.8738,5.0769,70.00,48,100,3.53,NG,1612657862
560,inuvik,68.3499,-133.7218,-18.40,76,100,4.61,CA,1612657862
561,adrasmon,40.6472,69.9856,46.40,66,40,17.90,TJ,1612657862


### Humidity Heatmap

In [7]:
#configure gmaps
gmaps.configure(api_key=g_key)

#create call list for locations and humidity
coordinates = weather_data[["Lat", "Lon"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [8]:
#Create gmap and add heat layer.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
# Add a "Hotel Name" column to the DataFrame.
weather_data["Hotel Name"] = ""


hotel = weather_data.loc[(
    weather_data["Maximum Temp"] > 60)]
hotel = hotel.loc[(
    hotel["Maximum Temp"] < 75)]

#reduce dataframe to select columns
hotel = hotel[["City", "Country", "Lat", "Lon", "Hotel Name"]]
hotel = hotel.dropna(how='any')
hotel

,City,Country,Lat,Lon,Hotel Name
8,obo,CF,5.3998,26.4869,
10,richards bay,ZA,-28.7830,32.0377,
13,sao francisco de paula,BR,-29.4481,-50.5836,
16,bluff,NZ,-46.6000,168.3333,
20,castro,BR,-24.7911,-50.0119,
...,...,...,...,...,...
529,sucua,EC,-2.4667,-78.1667,
531,zemio,CF,5.0314,25.1361,
544,mazatlan,MX,23.2167,-106.4167,
548,waipawa,NZ,-41.4122,175.5153,


In [61]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel["City"])):

    lat = hotel["Lat"]
    lon = hotel["Lon"]

    coordinates = zip(lat, lon)

    params = {
        "location": coordinates, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except Exception:
        pass
# Dataframe with nearest hotel
hotels

[]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
